In [1]:
import os, re, csv
import datetime
from itertools import islice
from tqdm.auto import tqdm
from collections import defaultdict

In [2]:
stoplist = ['и', 'в', 'не', 'на', 'я', 'быть', 'он', 'с', 'что', 'а', 'по', 'это', 'она', 'этот', 'к',
      'но', 'они', 'мы', 'как', 'из', 'у', 'который', 'то', 'за', 'свой', 'что', 'весь', 'год', 'от', 'так', 'о',
      'для', 'ты', 'же', 'все', 'тот', 'мочь', 'вы', 'человек', 'такой', 'его', 'сказать', 'только', 'или',
      'еще', 'бы', 'себя', 'один', 'как', 'уже', 'до', 'время', 'если', 'сам', 'когда', 'другой', 'вот',
      'говорить', 'наш', 'мой', 'знать', 'стать', 'при', 'чтобы', 'дело', 'жизнь', 'кто', 'первый', 'очень', 'два',
      'день', 'ее', 'новый', 'рука', 'даже', 'во', 'со', 'раз', 'где', 'там', 'под', 'можно', 'ну', 'какой',
      'после', 'их', 'работа', 'без', 'самый', 'потом', 'надо', 'хотеть', 'ли', 'слово', 'идти', 'большой', 'должен',
      'место', 'иметь', 'ничто', 'то', 'сейчас', 'тут', 'лицо', 'каждый', 'друг', 'нет', 'теперь', 'ни', 'глаз', 'тоже',
      'тогда', 'видеть', 'вопрос', 'через', 'да', 'здесь', 'дом', 'да', 'потому', 'сторона', 'какой-то', 'думать', 'сделать',
      'один', 'два', 'три', 'четыре', 'пять', 'шесть', 'семь', 'восесь', 'девять', 'десять', 'одиннадцать', 'двенадцать', 'двадцать',
      'трирдцать', 'сорок', 'пятьдесят', 'шестьдесят', 'семьдесят', 'восемьдесят', 'девяностно', 'сто', 'двести', 'триста',
      'четыреста', 'пятьсот', 'шестьсот', 'семьсот', 'восемьсот', 'девятьсот', 'тысяча', 'миллион', 'январь', 'февраль',
      'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь', 'понедельник', 'вторник',
      'среда', 'четверг', 'пятница', 'суббота', 'воскресенье', 'какой-то', 'чей', 'гораздо', 'мало', 'жизнь', 'отмечать',
      'принимать', 'также', 'очень', 'мочь', 'рассказывать', 'происходить', 'день', 'человек', 'совершать', 'иметь', 'суметь',
      'несмотря', 'пока', 'рано', 'попадать', 'спустя', 'знать', 'го', 'гг', 'двое', 'трое', 'неделя', 'месяц', 'год', 'первый',
      'второй', 'третий', 'идти', 'сказать', 'источник', 'час', 'затем', 'время', 'никто', 'сразу', 'часто', 'из-под', 'пора',
      'проходить', 'смочь', 'четверо', 'ничто', 'общий', 'вновь', 'кроме', 'иной', 'хотеть', 'создавать', 'становиться',
      'находиться', 'вне', 'хотя', 'нужно', 'крепко', 'пройти', 'вести', 'сотый', 'доля', 'поэтому', 'часть', 'ждать', 'слишком',
      'низко', 'плохо', 'видно', 'все-таки', 'стоить', 'вокруг', 'вместе', 'рука', 'известный', 'крайне', 'вряд', 'удаваться',
      'столько', 'сколько', 'задолго', 'сугубо', 'во-первых', 'во-вторых', 'ввиду', 'регулярно', 'публично', 'кто-либо', 'чрезвычайно',
            'чрезвычайно',
           'in', 'the', 'of', 'this', 'is', 'xxi', 'xx', 's', 'don t', 'can t', 'n', 'b', 'z']

names = '''Алиса, Александра, Алёна, Алина, Алла, Анастасия, Анжелика, Анна, Валентина, Валерия, Вера, Вероника, Виктория,
Галина, Дарья, Диана, Ева, Ева, Евгения, Екатерина, зоя, Алёна, Елена, Жанна, инесса, Инна, Ирина, Карина, Кристина, Ксения,
Лариса, Любовь, Людмила, Маргарита, Марина, Мария, Мила, Милана, Надежда, Наталья, Ника, Нина, Оксана, Олеся, Ольга, Полина,
рита, Руслана, Сания, сара, Светлана, София, Софья, Тамара, Татьяна, ульяна, Юлия, Яна,
Александр, Алексей, Анатолий, Андрей, Антон, Аркадий, Арсений, Артём, Артур, Борис, Вадим, Валентин, Валерий, Василий,
Виктор, Виталий, Владимир, Владислав, Вячеслав, Георгий, Глеб, Григорий, Даниил, Денис, Дмитрий, Евгений, Егор, жан, Иван,
Игорь, Илья, Кирилл, Константин, кузя, Лев, Леонид, Максим, марат, Марк, Матвей, Михаил, Никита, Николай, Олег, Павел, Пётр, Роман, Руслан,
Рустам, Сергей, Степан, Тимур, Федор, эдуард, эмиль, Юрий, Ярослав, омар'''.replace('ё', 'е')
names = names.lower().replace(' ', '').replace('\n', '').split(',')
stoplist += names

In [3]:
year = 2013
with open(f'rises_u{year}.tsv', encoding = 'utf-8') as f:
    with open(f'filtered-rises_u{year}.tsv', 'w', encoding='utf-8') as new:
        for line in f.readlines():
            token = line.split('\t')[0]
            if not token.isdigit() and not token.replace('-','').isdigit()\
            and re.search('[A-z]', token) is None and len(token)>2\
            and token not in stoplist:
                new.write(line)
                
with open(f'rises_b{year}.tsv', encoding='utf-8') as f:
    bis = f.read()
    with open(f'filtered-rises_b{year}.tsv', 'w', encoding='utf-8') as w:
        for line in bis.splitlines():
            bigram, _ = line.split('\t', 1)
            tokens = bigram.split(' ')
            if any(it in stoplist for it in tokens) or any(it.isdigit() for it in tokens)\
            or token.replace('-','').replace(' ', '').isdigit():
                pass
            else:
                w.write(f'{line}\n')

In [4]:
periods = defaultdict(list)

with open(f'filtered-rises_u{year}.tsv', encoding='utf-8') as f:
    for line in f.readlines():
        token = line.split('\t')[0]
        perio = line.strip('\n').split('\t')[1:]
        #yeared = []
        for period in perio:
            periods[f'{period}'].append(token)
        
with open(f'filtered-rises_b{year}.tsv', encoding='utf-8') as f:
    for line in f.readlines():
        token = line.split('\t')[0]
        perio = line.strip('\n').split('\t')[1:]
        #yeared = []
        for period in perio:
            periods[f'{period}'].append(token)
 
        
periods = dict(periods)

with open(f'periods-both{year}.tsv', 'w', encoding='utf-8') as f:
    for period in sorted(periods.keys()):
        per = period
        tokens = sorted(periods[period])
        toks = "\t".join(tokens)
        f.write(f'{period}\t{toks}\n')

#### поиск

In [36]:
year = 2013

In [37]:
with open(f'periods-both{year}.tsv', encoding='utf-8') as f:
    period_dict = {}
    for line in f.readlines():
        period, topics = line.split('\t', 1)
        period_dict[period] = topics
        
stihiru = '''здравствовать, произведение, ведущий, конкурс, балл, стихобалл, взнос, приз, б, пзс, зрительский, симпатия, литературно-художественный, антология, прочтение, заявка, внимание, рецензия'''
stihiru = stihiru.lower().replace(' ', '').replace('\n', '').split(',')
stihiru.append('уважаемый автор')

In [23]:
lag = 10
with open(f'found_news{year}.tsv', 'w', encoding='utf-8') as found:
    for period in tqdm(period_dict.keys()):
        dates = []
        months = []
        for i in range(lag):
            new_date = datetime.datetime.strptime(period, '%Y-%m-%d') + datetime.timedelta(i)
            string_date = new_date.strftime('%Y\t%m\t%d')
            dates.append(string_date)
            months.append(new_date.strftime('%m'))
        print(dates)
        months = set(months)
        for month in months:  # достать все строки из этих дат, в каждой строке из этих дат найти все ключи из дат
            line_nums = []  # списком ключи, потом стихотворение в строчку \n
            #poemfile = os.path.join('F:/c21/poems_lemmatized', f'mystem-{year}-{month}.tsv') 
            with open(f'{year}lentavzglyad.tsv', encoding='utf-8') as t:
               # filename = f'F:\c21\filtered_raw\filtered-{year}-{month}.txt'
                #filename = os.path.join('F:/c21/filtered_raw', f'filtered-{year}-{month}.txt')
                with open(f'{year}raw.tsv', encoding='utf-8') as r:
                    rawreader = r.readlines()
                    datareader = csv.reader(t, delimiter='\t')
                    #print(period, dates, period_dict[period])
                    for num, row in enumerate(datareader):
                        line_date = f'{row[0]}\t{row[1]}\t{row[2]}'
                        if line_date in dates:
                            found_tokens = []
                            tokens =set(row[3].replace('\n', '').strip(' ').split(' '))
                            for token in tokens:
                                if token in period_dict[period].split('\t'):
                                    found_tokens.append(token)
                            if len(found_tokens) >= 3:
                               # print(found_tokens)
                                #poem = rawreader[num].split('\t')[3]
                               # poem_date = '\t'.join(rawreader[num].split('\t')[:3])
                                newsline = rawreader[num].split('\t')
                                date = '\t'.join(newsline[:3])
                                news = '\t'.join(newsline[3:5])
                                found.write(f'{date}\t{found_tokens}\t{news}')

['2013\t01\t21', '2013\t01\t22', '2013\t01\t23', '2013\t01\t24', '2013\t01\t25', '2013\t01\t26', '2013\t01\t27', '2013\t01\t28', '2013\t01\t29', '2013\t01\t30']
['2013\t01\t31', '2013\t02\t01', '2013\t02\t02', '2013\t02\t03', '2013\t02\t04', '2013\t02\t05', '2013\t02\t06', '2013\t02\t07', '2013\t02\t08', '2013\t02\t09']
['2013\t02\t10', '2013\t02\t11', '2013\t02\t12', '2013\t02\t13', '2013\t02\t14', '2013\t02\t15', '2013\t02\t16', '2013\t02\t17', '2013\t02\t18', '2013\t02\t19']
['2013\t02\t20', '2013\t02\t21', '2013\t02\t22', '2013\t02\t23', '2013\t02\t24', '2013\t02\t25', '2013\t02\t26', '2013\t02\t27', '2013\t02\t28', '2013\t03\t01']
['2013\t03\t02', '2013\t03\t03', '2013\t03\t04', '2013\t03\t05', '2013\t03\t06', '2013\t03\t07', '2013\t03\t08', '2013\t03\t09', '2013\t03\t10', '2013\t03\t11']
['2013\t03\t12', '2013\t03\t13', '2013\t03\t14', '2013\t03\t15', '2013\t03\t16', '2013\t03\t17', '2013\t03\t18', '2013\t03\t19', '2013\t03\t20', '2013\t03\t21']
['2013\t03\t22', '2013\t03\t23', '

In [13]:
# convert news dataset to unified format
with open(f'{year}newsraw.tsv', encoding='utf-8') as r:
    with open(f'{year}raw.tsv', 'w', encoding='utf-8') as w:
        lines = r.readlines()
        for line in lines:
            if line.startswith('http'):
                newsline = line.strip('\n').replace('\tlenta', '')
                t = newsline.split(',')
                news = ','.join(t[2:-1])
                date = t[-1].replace('/', '\t')
                w_line = f'{date}\tlenta\t{news}\n'
                w.write(w_line)
            else:
                items = line.split('\t')
                news = items[4].strip('\n')
                date = '\t'.join(items[0:3])
                w.write(f'{date}\t{news}\t{items[3]}\n')